# 🔒 FraudGuard Training Notebook

**AD-RL-GNN Fraud Detection** | Full training pipeline with mini-batch processing

This notebook trains the FraudGuard model on the IEEE-CIS fraud detection dataset using:
- **NeighborLoader** for memory-efficient mini-batch training
- **FAISS** for similarity graph construction (GPU if available, CPU fallback)
- **FocalLoss** for class-imbalanced learning

**Target Metrics:**
- Specificity: 98.72%
- G-Means Improvement: 18.11%
- P95 Latency: <100ms

## 1️⃣ Setup Environment

In [ ]:
# Mount Google Drive for data storage
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Clone repository
!git clone https://github.com/govind104/fraudguard.git
%cd fraudguard

In [ ]:
# Install dependencies
# Note: faiss-gpu may not be available on Python 3.12
# The code will fallback to faiss-cpu automatically
# GNN training STILL runs on GPU - only graph building uses CPU FAISS
!pip install -q torch torch-geometric pandas numpy scikit-learn pyyaml structlog

# Try faiss-gpu first, fallback to faiss-cpu
import subprocess
result = subprocess.run(['pip', 'install', '-q', 'faiss-gpu'], capture_output=True)
if result.returncode != 0:
    print('⚠️ faiss-gpu not available, using faiss-cpu')
    print('   (Graph building on CPU, but GNN training still runs on GPU!)')
    !pip install -q faiss-cpu
else:
    print('✓ faiss-gpu installed')

import torch

# 1. Get exact versions
pt_version = torch.__version__.split('+')[0]  # e.g., 2.5.1
cuda_version = "cu" + torch.version.cuda.replace('.', '')  # e.g., cu124
wheel_url = f"https://data.pyg.org/whl/torch-{pt_version}+{cuda_version}.html"

print(f"PyTorch: {pt_version}, CUDA: {cuda_version}")
print(f"Downloading from: {wheel_url}")

# 2. Install with visible output (force reinstall to fix broken partial installs)
!pip install --force-reinstall torch-scatter torch-sparse -f $wheel_url

# Install repo in editable mode
!pip install -e .

print('\n✓ Environment setup complete')

## 2️⃣ Configuration

In [ ]:
import os

# ==============================================
# CONFIGURATION - UPDATE THESE PATHS AS NEEDED
# ==============================================

# Data paths - Point to your Google Drive folders
DATA_DIR = "/content/drive/MyDrive/ieee-fraud-detection"
MODELS_DIR = "/content/drive/MyDrive/fraudguard-models"
LOGS_DIR = "/content/drive/MyDrive/fraudguard-logs"

# Training parameters
SAMPLE_FRAC = 1.0      # Use full dataset (1.0 = 100%)
MAX_EPOCHS = 100
BATCH_SIZE = 4096      # Reduce to 2048 or 1024 if OOM
NUM_NEIGHBORS = [25, 10]  # 2-hop neighborhood sampling

# Create directories
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(LOGS_DIR, exist_ok=True)

print(f"Data: {DATA_DIR}")
print(f"Models: {MODELS_DIR}")
print(f"Logs: {LOGS_DIR}")
print(f"\nBatch size: {BATCH_SIZE}")
print(f"Sample fraction: {SAMPLE_FRAC*100:.0f}%")

## 3️⃣ Verify GPU and FAISS

In [ ]:
import torch
import faiss

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print("\n✓ GNN training will run on GPU")
else:
    print("\n⚠️ WARNING: No GPU detected. Go to Runtime > Change runtime type > GPU")

# Check FAISS GPU
faiss_gpus = faiss.get_num_gpus() if hasattr(faiss, 'get_num_gpus') else 0
print(f"\nFAISS GPUs: {faiss_gpus}")
if faiss_gpus == 0:
    print("   (Using CPU FAISS for graph building - this is OK)")

## 4️⃣ Load and Preprocess Data

In [ ]:
import sys
sys.path.insert(0, '/content/fraudguard')

from pathlib import Path
from src.data.loader import FraudDataLoader
from src.data.preprocessor import FeaturePreprocessor
from src.data.graph_builder import GraphBuilder
from src.utils.config import load_data_config
from src.utils.device_utils import set_seed, get_device

set_seed(42)
device = get_device()
print(f"Using device: {device}")

# Load config and override path with notebook variable
data_cfg = load_data_config()
data_cfg.paths.raw_data_dir = Path(DATA_DIR)

# Load data with corrected path
loader = FraudDataLoader(config=data_cfg)
df = loader.load_train_data(sample_frac=SAMPLE_FRAC)
train_df, val_df, test_df = loader.create_splits(df)

print(f"\nData loaded:")
print(f"  Train: {len(train_df):,}")
print(f"  Val: {len(val_df):,}")
print(f"  Test: {len(test_df):,}")
print(f"  Fraud rate: {df['isFraud'].mean()*100:.2f}%")

## 5️⃣ Build or Load Graph

In [ ]:
import torch
import numpy as np
import faiss
import gc
import os
from src.utils.config import load_model_config

GRAPH_CACHE = f"{MODELS_DIR}/edges_full.pt"
FEATURES_CACHE = f"{MODELS_DIR}/features_full.pt"

# Configuration
CHUNK_SIZE = 50000        
THRESHOLD = 0.90          
MAX_NEIGHBORS = 50        

def build_edges_in_chunks(source_feats, target_feats, index, start_offset_idx, desc="Processing"):
    """
    Searches 'index' using 'source_feats' in chunks to avoid OOM.
    Returns a list of edge tensors (2, num_edges).
    """
    edge_chunks = []
    num_source = len(source_feats)
    
    for start in range(0, num_source, CHUNK_SIZE):
        end = min(start + CHUNK_SIZE, num_source)
        print(f"  [{desc}] Batch {start}-{end} / {num_source}...")
        
        # 1. Get batch
        batch_feats = source_feats[start:end]
        
        # 2. Search FAISS
        D, I = index.search(batch_feats, MAX_NEIGHBORS)
        
        # 3. Vectorized Filter
        src_indices = np.arange(start, end) + start_offset_idx
        src_expanded = src_indices[:, None]
        
        # Create mask
        mask = (D > THRESHOLD) & (I != src_expanded) & (I != -1)
        
        # Broadcast src to match shape
        src_broadcast = np.broadcast_to(src_expanded, I.shape)
        
        valid_src = src_broadcast[mask]
        valid_dst = I[mask]
        
        if len(valid_src) > 0:
            chunk_edges = torch.stack([
                torch.from_numpy(valid_src),
                torch.from_numpy(valid_dst)
            ], dim=0).to(torch.long)
            edge_chunks.append(chunk_edges)
            
        del D, I, mask, batch_feats, valid_src, valid_dst, src_broadcast
        gc.collect()
        
    return edge_chunks

# ==========================================
# Main Execution Flow
# ==========================================

if os.path.exists(GRAPH_CACHE):
    print("Loading cached graph from Google Drive...")
    edge_index = torch.load(GRAPH_CACHE)
    print(f"Loaded {edge_index.shape[1]:,} edges")
    
    if 'X_full' not in locals():
         X_full = torch.load(FEATURES_CACHE)
    
    edge_index = edge_index.to(device)
    X_full = X_full.to(device)

else:
    print("🚀 Starting Memory-Optimized Graph Build (Directed)...")
    
    # 1. Prepare Data
    prep = FeaturePreprocessor()
    X_train_np = prep.fit_transform(train_df).cpu().numpy().astype(np.float32)
    X_val_np = prep.transform(val_df).cpu().numpy().astype(np.float32)
    X_test_np = prep.transform(test_df).cpu().numpy().astype(np.float32)
    
    # Save lengths
    n_train = len(X_train_np)
    n_val = len(X_val_np)
    n_test = len(X_test_np)
    
    # Save full features
    X_full = torch.cat([
        torch.from_numpy(X_train_np), 
        torch.from_numpy(X_val_np), 
        torch.from_numpy(X_test_np)
    ])
    
    # Clear DataFrames
    del df, train_df, val_df, test_df
    gc.collect()

    # 2. Setup FAISS (CPU)
    print("Building FAISS Index on Train Data...")
    faiss.normalize_L2(X_train_np)
    index = faiss.IndexFlatIP(X_train_np.shape[1])
    index.add(X_train_np)
    print("✓ FAISS index built (CPU mode)")

    # 3. Build Edges (Phased)
    print("\nPhase 1: linking Train -> Train...")
    train_chunks = build_edges_in_chunks(X_train_np, None, index, 0, "Train")
    del X_train_np; gc.collect()

    print("\nPhase 2: linking Val -> Train...")
    faiss.normalize_L2(X_val_np)
    val_chunks = build_edges_in_chunks(X_val_np, None, index, n_train, "Val")
    del X_val_np; gc.collect()

    print("\nPhase 3: linking Test -> Train...")
    faiss.normalize_L2(X_test_np)
    test_chunks = build_edges_in_chunks(X_test_np, None, index, n_train + n_val, "Test")
    del X_test_np; gc.collect()

    # 4. Merge (Directed Graph Only)
    print("\nMerging edge chunks...")
    all_chunks = train_chunks + val_chunks + test_chunks
    
    if len(all_chunks) > 0:
        edge_index = torch.cat(all_chunks, dim=1)
        
        # --- OOM FIX: SKIPPING SYMMETRIZATION ---
        # We perform a lightweight sort instead of a heavy unique/symmetrize
        # This keeps the graph directed (A->B) but saves 50% RAM
        print("Sorting edges (Directed)...")
        # Sort by source node (row 0)
        idx = edge_index[0].argsort()
        edge_index = edge_index[:, idx]
        
    else:
        edge_index = torch.empty((2, 0), dtype=torch.long)

    print(f"Total Edges Generated: {edge_index.shape[1]:,}")
    
    # Save to Drive
    torch.save(edge_index, GRAPH_CACHE)
    torch.save(X_full, FEATURES_CACHE)
    print(f"✓ Saved to {GRAPH_CACHE}")
    
    del index, all_chunks
    gc.collect()

# Final Device Move
print("Moving final tensors to GPU...")
X_full = X_full.to(device)
edge_index = edge_index.to(device)
print(f"\nFinal Graph ready on {device}")

## 6️⃣ Setup Mini-Batch Training

In [ ]:
from torch_geometric.loader import NeighborLoader
from torch_geometric.data import Data
from pathlib import Path
import torch.nn.functional as F
import gc
import sys

# 1. Reload Data briefly to get Labels & Lengths (since we deleted them)
# We need to re-import loader components if they were lost
sys.path.insert(0, '/content/fraudguard')
from src.data.loader import FraudDataLoader
from src.utils.config import load_data_config

print("Reloading data to extract labels...")
data_cfg = load_data_config()
data_cfg.paths.raw_data_dir = Path(DATA_DIR) # Ensure pointing to Drive
loader = FraudDataLoader(config=data_cfg)

# Load and split
df_temp = loader.load_train_data(sample_frac=1.0) # Use same sample_frac!
train_df, val_df, test_df = loader.create_splits(df_temp)

# 2. Extract Labels & Sizes
print("Extracting labels...")
train_labels = torch.tensor(train_df["isFraud"].values, dtype=torch.long)
val_labels = torch.tensor(val_df["isFraud"].values, dtype=torch.long)
test_labels = torch.tensor(test_df["isFraud"].values, dtype=torch.long)

n_train = len(train_df)
n_val = len(val_df)
n_test = len(test_df)

# 3. Aggressive Cleanup (Free RAM immediately)
del df_temp, train_df, val_df, test_df
gc.collect()
print("Dataframes deleted to free RAM.")

# 4. Prepare Masks & Labels
all_labels = torch.cat([train_labels, val_labels, test_labels]).to(device)

n_total = n_train + n_val + n_test
train_mask = torch.zeros(n_total, dtype=torch.bool)
val_mask = torch.zeros(n_total, dtype=torch.bool)
test_mask = torch.zeros(n_total, dtype=torch.bool)

# Set masks using the calculated lengths
train_mask[:n_train] = True
val_mask[n_train : n_train + n_val] = True
test_mask[n_train + n_val :] = True

print(f"Masks created: Train={train_mask.sum()}, Val={val_mask.sum()}, Test={test_mask.sum()}")

# 5. Create PyG Data object
# Ensure X_full and edge_index are on the correct device
if X_full.device != device:
    X_full = X_full.to(device)
if edge_index.device != device:
    edge_index = edge_index.to(device)

data = Data(x=X_full, edge_index=edge_index, y=all_labels)
data.train_mask = train_mask
data.val_mask = val_mask
data.test_mask = test_mask

# 6. Create NeighborLoaders
print(f"Initializing NeighborLoaders (Batch Size: {BATCH_SIZE})...")

train_loader = NeighborLoader(
    data,
    num_neighbors=NUM_NEIGHBORS,  # [25, 10]
    batch_size=BATCH_SIZE,
    input_nodes=train_mask,
    shuffle=True
)

val_loader = NeighborLoader(
    data,
    num_neighbors=NUM_NEIGHBORS,
    batch_size=BATCH_SIZE,
    input_nodes=val_mask,
    shuffle=False
)

print(f"✓ Train batches: {len(train_loader)}")
print(f"✓ Val batches: {len(val_loader)}")

## 7️⃣ Train Model

In [ ]:
from src.models import FraudGNN, FocalLoss, compute_class_weights
from sklearn.metrics import confusion_matrix, f1_score
import numpy as np
import time

# Initialize model
model = FraudGNN(in_channels=X_full.shape[1]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)
weights = compute_class_weights(train_labels, device)
criterion = FocalLoss(alpha=0.5, gamma=4, weight=weights)

# Training config
best_gmeans = 0
patience = 30
patience_counter = 0
history = []

print("Starting training...\n")
print(f"{'Epoch':>5} | {'Loss':>8} | {'Spec':>8} | {'Recall':>8} | {'F1':>8} | {'G-Means':>8}")
print("-" * 60)

start_time = time.time()

for epoch in range(MAX_EPOCHS):
    # Training
    model.train()
    total_loss = 0
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        loss = criterion(out[:batch.batch_size], batch.y[:batch.batch_size])
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    
    # Validation every 5 epochs
    if epoch % 5 == 0:
        model.eval()
        all_preds, all_true = [], []
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(device)
                out = model(batch.x, batch.edge_index)
                pred = out[:batch.batch_size].argmax(dim=1)
                all_preds.extend(pred.cpu().numpy())
                all_true.extend(batch.y[:batch.batch_size].cpu().numpy())
        
        cm = confusion_matrix(all_true, all_preds, labels=[0, 1])
        tn, fp, fn, tp = cm.ravel()
        tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
        tnr = tn / (tn + fp) if (tn + fp) > 0 else 0
        gmeans = np.sqrt(tpr * tnr)
        f1 = f1_score(all_true, all_preds, zero_division=0)
        
        history.append({'epoch': epoch, 'loss': avg_loss, 'spec': tnr, 'recall': tpr, 'f1': f1, 'gmeans': gmeans})
        
        print(f"{epoch+1:>5} | {avg_loss:>8.4f} | {tnr*100:>7.2f}% | {tpr*100:>7.2f}% | {f1*100:>7.2f}% | {gmeans*100:>7.2f}%")
        
        # Early stopping
        if gmeans > best_gmeans:
            best_gmeans = gmeans
            patience_counter = 0
            torch.save(model.state_dict(), f"{MODELS_DIR}/best_model.pt")
        else:
            patience_counter += 1
            if patience_counter >= patience // 5:
                print(f"\nEarly stopping at epoch {epoch+1}")
                break

train_time = time.time() - start_time
print(f"\nTraining complete in {train_time/60:.1f} minutes")
print(f"Best validation G-Means: {best_gmeans*100:.2f}%")

## 8️⃣ Evaluate on Test Set

In [ ]:
import time

# Load best model
model.load_state_dict(torch.load(f"{MODELS_DIR}/best_model.pt"))
model.eval()

# Full neighborhood for test evaluation
test_loader = NeighborLoader(
    data,
    num_neighbors=[-1, -1],  # Full neighborhood
    batch_size=BATCH_SIZE,
    input_nodes=test_mask,
    shuffle=False,
)

all_preds, all_true = [], []
latencies = []

with torch.no_grad():
    for batch in test_loader:
        batch = batch.to(device)
        start = time.perf_counter()
        out = model(batch.x, batch.edge_index)
        latencies.append((time.perf_counter() - start) * 1000)
        pred = out[:batch.batch_size].argmax(dim=1)
        all_preds.extend(pred.cpu().numpy())
        all_true.extend(batch.y[:batch.batch_size].cpu().numpy())

# Compute metrics
cm = confusion_matrix(all_true, all_preds, labels=[0, 1])
tn, fp, fn, tp = cm.ravel()
tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
tnr = tn / (tn + fp) if (tn + fp) > 0 else 0
gmeans = np.sqrt(tpr * tnr)
f1 = f1_score(all_true, all_preds, zero_division=0)

print("=" * 60)
print("FINAL TEST RESULTS")
print("=" * 60)
print(f"\nConfusion Matrix:")
print(f"  TP: {tp:,}  |  FN: {fn:,}")
print(f"  FP: {fp:,}  |  TN: {tn:,}")
print(f"\nPerformance:")
print(f"  Specificity:  {tnr*100:.2f}%  (CV target: 98.72%)")
print(f"  Recall:       {tpr*100:.2f}%")
print(f"  F1 Score:     {f1*100:.2f}%")
print(f"  G-Means:      {gmeans*100:.2f}%")
print(f"\nLatency:")
print(f"  Mean: {np.mean(latencies):.1f}ms")
print(f"  P95:  {np.percentile(latencies, 95):.1f}ms  (CV target: <100ms)")
print(f"  P99:  {np.percentile(latencies, 99):.1f}ms")

## 9️⃣ Save Final Model

In [ ]:
# Save final model with metrics
torch.save({
    "model_state_dict": model.state_dict(),
    "config": {
        "in_channels": X_full.shape[1],
        "specificity": tnr,
        "recall": tpr,
        "gmeans": gmeans,
        "f1": f1,
    },
    "history": history,
}, f"{MODELS_DIR}/fraudguard_final.pt")

print(f"✓ Model saved to {MODELS_DIR}/fraudguard_final.pt")
print(f"✓ Best model saved to {MODELS_DIR}/best_model.pt")

## 🔟 CV Claims Comparison

In [ ]:
# CV Claims comparison
CV_CLAIMS = {
    "specificity": 98.72,
    "gmeans_improvement": 18.11,
    "p95_latency_ms": 100,
}

achieved_spec = tnr * 100
p95_latency = np.percentile(latencies, 95)

print("=" * 60)
print("CV CLAIMS COMPARISON")
print("=" * 60)
print(f"| {'Metric':<20} | {'Achieved':>12} | {'CV Claim':>12} | {'Status':>6} |")
print(f"|{'-'*22}|{'-'*14}|{'-'*14}|{'-'*8}|")

# Specificity
status_spec = "✓" if abs(achieved_spec - CV_CLAIMS['specificity']) <= 3 else "✗"
print(f"| {'Specificity':<20} | {achieved_spec:>11.2f}% | {CV_CLAIMS['specificity']:>11.2f}% | {status_spec:>6} |")

# Latency
status_lat = "✓" if p95_latency < CV_CLAIMS['p95_latency_ms'] else "✗"
print(f"| {'P95 Latency':<20} | {p95_latency:>10.1f}ms | {'<100':>10}ms | {status_lat:>6} |")

print("=" * 60)
print("\n✓ = PASS (within tolerance) | ✗ = INVESTIGATE")